In [1]:
import corrcal
import numpy as np
import matplotlib.pyplot as plt
import ctypes
import numba

The first goal in this notebook is to investigate the `corrcal.cfuncs.apply_gains_to_matrix` function (`apply_gains_to_mat` function in the C code). This involves the following:

* Simulate some data and build the covariance matrix.
* Simulate per-antenna gains.
* Apply the gains using `numpy`, basic python (with and without `numba`), and `corrcal`.
* Compare the results.

In [2]:
# First, make the antenna arrays.
ant1 = np.array([0, 1])
ant2 = np.array([1, 2])
ants = set(ant1).union(set(ant2))

# Make the gains here because it's easy.
gains = np.array(
    list(
        np.random.uniform() + 1j * np.random.uniform()
        for ant in ants
    )
)

# Simulate the data
data = np.array(
    list(
        np.random.uniform() + 1j * np.random.uniform()
        for ant in ant1
    )
)

# Make the complex-valued covariance
cov = np.outer(data, np.conj(data))

In [4]:
# Step 1: brute force
def apply_gains_simple(mat, gains, ant1, ant2):
    out = np.zeros_like(mat)
    for i, ai in enumerate(ant1):
        for j, aj in enumerate(ant2):
            out[i,j] = mat[i,j] * gains[ai] * np.conj(gains[aj])
    return out

# Step 2: numba-fied brute force
@numba.njit
def apply_gains_numba(mat, gains, ant1, ant2):
    out = np.zeros_like(mat)
    for i, ai in enumerate(ant1):
        for j, aj in enumerate(ant2):
            out[i,j] = mat[i,j] * gains[ai] * np.conj(gains[aj])
    return out

# Step 3: corrcal version
def apply_gains_corrcal(mat, gains, ant1, ant2):
    out = mat.copy()
    corrcal.cfuncs.apply_gains_to_matrix(
        out.ctypes.data,
        gains.ctypes.data,
        ant1.ctypes.data,
        ant2.ctypes.data,
        len(ant1),
        len(ant1),
    )
    return out

In [5]:
# Check that all results are equivalent
simple_numba_match = np.allclose(
    apply_gains_simple(cov, gains, ant1, ant2),
    apply_gains_numba(cov, gains, ant1, ant2),
)
simple_corrcal_match = np.allclose(
    apply_gains_simple(cov, gains, ant1, ant2),
    apply_gains_corrcal(cov, gains, ant1, ant2),
)
simple_numba_match, simple_corrcal_match

(True, False)

In [6]:
apply_gains_simple(cov, gains, ant1, ant2)

array([[0.0428176 -0.02820719j, 0.06377239-0.01017758j],
       [0.02621248+0.01261466j, 0.02354736+0.02807006j]])

In [7]:
apply_gains_corrcal(cov, gains, ant1, ant2)

array([[0.0428176 -0.02820719j, 0.03939508+0.01779453j],
       [0.04306916-0.00580549j, 0.02354736+0.02807006j]])

In [8]:
cov

array([[0.16902426+0.j        , 0.1290911 -0.06212464j],
       [0.1290911 +0.06212464j, 0.12142625+0.j        ]])

In [9]:
# Now do timing tests

In [10]:
%%timeit
_ = apply_gains_simple(cov, gains, ant1, ant2)

14.9 µs ± 676 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [11]:
%%timeit
_ = apply_gains_numba(cov, gains, ant1, ant2)

678 ns ± 45.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [12]:
%%timeit
_ = apply_gains_corrcal(cov, gains, ant1, ant2)

6.6 µs ± 166 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
